In [1]:
#Import some of the tools I know I will need:
import numpy as np
import pandas as pd
import requests
import zipfile
import io
from datetime import datetime
import random
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import pickle 

# Python 2 & 3 Compatibility
from __future__ import print_function, division

%matplotlib inline

In [2]:
pkl_filename_master = "/Users/torrie/Documents/Metis_Project_2_(Luther)/Past_Movie_detail_ALL_raw Pickle_file.pkl"
with open(pkl_filename_master, 'rb') as picklefile: 
    df_MovieData = pickle.load(picklefile)
    
df_MovieData = df_MovieData.reset_index(drop=True)

In [3]:
df_MovieData.columns

Index([u'Titles', u'Ratings', u'Duration', u'Release_date', u'Writer',
       u'Director', u'Actors', u'Genre', u'Budget', u'Opening_Weekend',
       u'Gross', u'Production_Companies', u'Metacritic_Score', u'IMDB_Rating',
       u'IMDB_rating_number'],
      dtype='object')

In [4]:
df_MovieData.dtypes

Titles                   object
Ratings                  object
Duration                 object
Release_date             object
Writer                   object
Director                 object
Actors                   object
Genre                    object
Budget                   object
Opening_Weekend          object
Gross                     int64
Production_Companies     object
Metacritic_Score         object
IMDB_Rating             float64
IMDB_rating_number        int64
dtype: object

Most of the data is "object" format, so I will need to convert dates to datetime and budgets and scores to numbers. 

In [5]:
df_MovieData.iloc[5]

Titles                                              When the Bough Breaks
Ratings                                                             PG-13
Duration                                                           PT107M
Release_date                                                   2016-09-09
Writer                                           [Jack Olsen, Jack Olsen]
Director                                                     [Jon Cassar]
Actors                  [Morris Chestnut, Regina Hall, Romany Malco, M...
Genre                                  [Drama, Horror, Mystery, Thriller]
Budget                                                           10000000
Opening_Weekend                                                  14202323
Gross                                                            29703843
Production_Companies                       [Screen Gems, Unique Features]
Metacritic_Score                                                       28
IMDB_Rating                           

Need to change ""'s to nan's , turn the duration into minutes, format dates as dates, 

In [6]:
df_MovieData['Duration'] = df_MovieData['Duration'].map(lambda x: x.lstrip('PT').rstrip('M'))
df_MovieData['Release_date'] = pd.to_datetime(df_MovieData['Release_date'], format='%Y-%m-%d', errors='ignore')
df_MovieData.replace('', np.nan, inplace=True)

In [7]:
def columns_to_numeric(df, column_list):
    for column_name in column_list:
        df[column_name] = pd.to_numeric(df[column_name], errors ='coerce')
        df[column_name]= df[column_name].fillna(0).astype(int)
        df[column_name] = df[column_name].astype(int) 

    return df

column_list = ['Budget','Opening_Weekend','Metacritic_Score', 'IMDB_rating_number' ]

df_MovieData = columns_to_numeric(df_MovieData, column_list)
        

In [8]:
df_MovieData.dtypes

Titles                          object
Ratings                         object
Duration                        object
Release_date            datetime64[ns]
Writer                          object
Director                        object
Actors                          object
Genre                           object
Budget                           int64
Opening_Weekend                  int64
Gross                            int64
Production_Companies            object
Metacritic_Score                 int64
IMDB_Rating                    float64
IMDB_rating_number               int64
dtype: object

In [ ]:
#df_MovieData[df_MovieData['Director'].str.len() > 1]['Director'].apply(lambda x: x[0].replace(',', ''))


In [10]:
def column_lists_to_variables(df, column_list, number_of_combinations):
    for column in column_list:
        #name1 = column + "_all"
        #df[name1] = df[column].replace(',','')
        #df[column + "len"] = df[column].apply(lambda x: len(x))
        for n in range(0,number_of_combinations):
            num = n+1
            df[column+"_"+str(num)] = np.nan
            df[column+"_"+str(num)] = df[df[column].str.len() > n][column].apply(lambda x: x[n])
            if n > 0:
                df[column+"_"+str(num)+"_comb"] = df[df[column].str.len() > n][column].apply(lambda x: x[:(n+1)])
            else:
                continue

    
    return df

def combination_variable(df, column1, column2, degree):
    
    return df

column_list = ['Director', 'Writer', "Actors", 'Production_Companies']
#del check
check = column_lists_to_variables(df_MovieData, column_list, 2)
check.columns

Index([u'Titles', u'Ratings', u'Duration', u'Release_date', u'Writer',
       u'Director', u'Actors', u'Genre', u'Budget', u'Opening_Weekend',
       u'Gross', u'Production_Companies', u'Metacritic_Score', u'IMDB_Rating',
       u'IMDB_rating_number', u'Director_1', u'Director_2', u'Director_2_comb',
       u'Writer_1', u'Writer_2', u'Writer_2_comb', u'Actors_1', u'Actors_2',
       u'Actors_2_comb', u'Production_Companies_1', u'Production_Companies_2',
       u'Production_Companies_2_comb'],
      dtype='object')

In [11]:
check.head(3)

,Titles,Ratings,Duration,Release_date,Writer,Director,Actors,Genre,Budget,Opening_Weekend,...,Director_2_comb,Writer_1,Writer_2,Writer_2_comb,Actors_1,Actors_2,Actors_2_comb,Production_Companies_1,Production_Companies_2,Production_Companies_2_comb
0,A Monster Calls,PG-13,108,2017-01-06,"[Patrick Ness, Patrick Ness]",[J.A. Bayona],"[Lewis MacDougall, Sigourney Weaver, Felicity ...","[Drama, Fantasy]",43000000,30909,...,NaN,Patrick Ness,Patrick Ness,"[Patrick Ness, Patrick Ness]",Lewis MacDougall,Sigourney Weaver,"[Lewis MacDougall, Sigourney Weaver]",Apaches Entertainment,La Trini,"[Apaches Entertainment, La Trini]"
1,A United Kingdom,PG-13,111,2017-02-10,"[Guy Hibbert, Susan Williams]",[Amma Asante],"[David Oyelowo, Rosamund Pike, Tom Felton, Jac...","[Biography, Drama, Romance]",0,66510,...,NaN,Guy Hibbert,Susan Williams,"[Guy Hibbert, Susan Williams]",David Oyelowo,Rosamund Pike,"[David Oyelowo, Rosamund Pike]",Pathé,BBC Films,"[Pathé, BBC Films]"
2,Carrie Pilby,NaN,98,2017-04-04,"[Kara Holden, Caren Lissner]",[Susan Johnson],"[Bel Powley, Nathan Lane, Gabriel Byrne, Vanes...","[Comedy, Drama]",0,13944,...,NaN,Kara Holden,Caren Lissner,"[Kara Holden, Caren Lissner]",Bel Powley,Nathan Lane,"[Bel Powley, Nathan Lane]",Braveart Films,NaN,NaN


What about doing dimensionality reduction on this one instead of only doing one-hot encoding like last time? 

Next: Get release season, Writer+director, genre, budget colums cleaned/made.